# Brutal force method to search for motifs


## Use python itertools

In [5]:
import itertools

print("itertools: 3 loops over 2 things")
for number in itertools.product(range(2), repeat=3):
    print(number, sum(2**(len(number)-i-1)*bit for i, bit in enumerate(number)))

print("\nitertools: 2 loops over 3 things")
for number in itertools.product(range(3), repeat=2):
    print(number)

print("\nitertools: permutations of mixed types")
for section in itertools.product(("I", "II", "III", "IV"),"ABC",range(1,3)):
    print(section)

itertools: 3 loops over 2 things
(0, 0, 0) 0
(0, 0, 1) 1
(0, 1, 0) 2
(0, 1, 1) 3
(1, 0, 0) 4
(1, 0, 1) 5
(1, 1, 0) 6
(1, 1, 1) 7

itertools: 2 loops over 3 things
(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)
(2, 0)
(2, 1)
(2, 2)

itertools: permutations of mixed types
('I', 'A', 1)
('I', 'A', 2)
('I', 'B', 1)
('I', 'B', 2)
('I', 'C', 1)
('I', 'C', 2)
('II', 'A', 1)
('II', 'A', 2)
('II', 'B', 1)
('II', 'B', 2)
('II', 'C', 1)
('II', 'C', 2)
('III', 'A', 1)
('III', 'A', 2)
('III', 'B', 1)
('III', 'B', 2)
('III', 'C', 1)
('III', 'C', 2)
('IV', 'A', 1)
('IV', 'A', 2)
('IV', 'B', 1)
('IV', 'B', 2)
('IV', 'C', 1)
('IV', 'C', 2)


# Bruteforce exact motif search

In [3]:
sequences = [
    'tagtggtcttttgagtgtagatccgaagggaaagtatttccaccagttcggggtcacccagcagggcagggtgacttaat',
    'cgcgactcggcgctcacagttatcgcacgtttagaccaaaacggagttagatccgaaactggagtttaatcggagtcctt',
    'gttacttgtgagcctggttagatccgaaatataattgttggctgcatagcggagctgacatacgagtaggggaaatgcgt',
    'aacatcaggctttgattaaacaatttaagcacgtagatccgaattgacctgatgacaatacggaacatgccggctccggg',
    'accaccggataggctgcttattagatccgaaaggtagtatcgtaataatggctcagccatgtcaatgtgcggcattccac',
    'tagatccgaatcgatcgtgtttctccctctgtgggttaacgaggggtccgaccttgctcgcatgtgccgaacttgtaccc',
    'gaaatggttcggtgcgatatcaggccgttctcttaacttggcggtgtagatccgaacgtctctggaggggtcgtgcgcta',
    'atgtatactagacattctaacgctcgcttattggcggagaccatttgctccactacaagaggctactgtgtagatccgaa',
    'ttcttacacccttctttagatccgaacctgttggcgccatcttcttttcgagtccttgtacctccatttgctctgatgac',
    'ctacctatgtaaaacaacatctactaacgtagtccggtctttcctgatctgccctaacctacaggtagatccgaaattcg']

def bruteForce(dna,k):
    """Finds a *k*-mer common to all sequences from a
       list of *dna* fragments with the same length"""
    M = len(dna)     # how many sequences
    N = len(dna[0])  # length of sequences
    for offset in itertools.product(range(N-k+1), repeat=M):    # iterate all possible combination of k-mers in M sequences
        for i in range(1,len(offset)):
            if dna[0][offset[0]:offset[0]+k] != dna[i][offset[i]:offset[i]+k]:
                break
        else:
            return offset, dna[0][offset[0]:offset[0]+10]

In [5]:
# consider the first M sequences 
# you can try a larger value of M, but be prepared to wait
M = 4       
position, motif = bruteForce(sequences[0:M], 10)
print(position, motif, '\n')

for i in range(M):
    p = position[i]
    print(sequences[i][:p]+sequences[i][p:p+10].upper()+sequences[i][p+10:])
print()

%time bruteForce(sequences[0:M], 10)

(17, 47, 18, 33) tagatccgaa 

tagtggtcttttgagtgTAGATCCGAAgggaaagtatttccaccagttcggggtcacccagcagggcagggtgacttaat
cgcgactcggcgctcacagttatcgcacgtttagaccaaaacggagtTAGATCCGAAactggagtttaatcggagtcctt
gttacttgtgagcctggtTAGATCCGAAatataattgttggctgcatagcggagctgacatacgagtaggggaaatgcgt
aacatcaggctttgattaaacaatttaagcacgTAGATCCGAAttgacctgatgacaatacggaacatgccggctccggg

Wall time: 5.07 s


((17, 47, 18, 33), 'tagatccgaa')

# Let's try again allowing for errors
Find a set of k-mers that maximize the consensus score.

In [1]:
def Score(s, DNA, k):
    """ 
        compute the consensus SCORE of a given k-mer alignment given 
        offsets into each DNA string. s = list of starting indices.
        DNA = list of nucleotide strings. k = Target Motif length
    """
    score = 0
    for i in range(k):
        # loop over string positions
        cnt = dict(zip("acgt",(0,0,0,0)))
        for j, sval in enumerate(s):
            base = DNA[j][sval+i] 
            cnt[base] += 1
        score += max(cnt.values())
    return score

def BruteForceMotifSearch(dna,k):
    M = len(dna)     # how many sequences
    N = len(dna[0])  # length of sequences
    bestScore = 0
    bestAlignment = []
    for offset in itertools.product(range(N-k+1), repeat=M):
        s = Score(offset,dna,k)
        if (s > bestScore):
            bestAlignment = [p for p in offset]
            bestScore = s
    print("best motif start positions:", bestAlignment, bestScore)
    for i in range(len(bestAlignment)):
        print(dna[i][bestAlignment[i]:bestAlignment[i]+k])

In [3]:
seqApprox = [
    'tagtggtcttttgagtgtagatctgaagggaaagtatttccaccagttcggggtcacccagcagggcagggtgacttaat',
    'cgcgactcggcgctcacagttatcgcacgtttagaccaaaacggagttggatccgaaactggagtttaatcggagtcctt',
    'gttacttgtgagcctggttagacccgaaatataattgttggctgcatagcggagctgacatacgagtaggggaaatgcgt',
    'aacatcaggctttgattaaacaatttaagcacgtaaatccgaattgacctgatgacaatacggaacatgccggctccggg',
    'accaccggataggctgcttattaggtccaaaaggtagtatcgtaataatggctcagccatgtcaatgtgcggcattccac',
    'tagattcgaatcgatcgtgtttctccctctgtgggttaacgaggggtccgaccttgctcgcatgtgccgaacttgtaccc',
    'gaaatggttcggtgcgatatcaggccgttctcttaacttggcggtgcagatccgaacgtctctggaggggtcgtgcgcta',
    'atgtatactagacattctaacgctcgcttattggcggagaccatttgctccactacaagaggctactgtgtagatccgta',
    'ttcttacacccttctttagatccaaacctgttggcgccatcttcttttcgagtccttgtacctccatttgctctgatgac',
    'ctacctatgtaaaacaacatctactaacgtagtccggtctttcctgatctgccctaacctacaggtcgatccgaaattcg']

%timeit Score([17, 47, 18, 33, 21, 0, 46, 70, 16, 65], seqApprox, 10)
%time BruteForceMotifSearch(seqApprox[0:3], 10)

19.6 µs ± 78.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


NameError: name 'itertools' is not defined

## Branch and Bound Motif Search
It takes too long to search all possible starting positions of k-mers. We can prune parts of the search tree if the most optimistic remaining positions cannot beat the current best score. 

In [6]:
bestAlignment = []
prunedPaths = 0

# Recursive Exploration of a Search Tree
def exploreMotifs(DNA,k,path,bestScore):
    """ Search for a k-length motif in the list of DNA sequences by exploring
        all paths in a search tree. Each call extends path by one. Once the
        path reaches the number of DNA strings a score is computed. """
    global bestAlignment, prunedPaths
    depth = len(path)
    M = len(DNA)
    if (depth == M):            # here we have an index in all M sequences
        s = Score(path,DNA,k)
        if (s > bestScore):
            bestAlignment = [p for p in path]
            return s
        else:
            return bestScore
    else:
        # Let's consider if an optimistic best score can beat the best score so far
        if (depth > 1):
            OptimisticScore = k*(M-depth) + Score(path,DNA,k)
        else:
            OptimisticScore = k*M
        if (OptimisticScore < bestScore):
            prunedPaths = prunedPaths + 1
            return bestScore
        else:
            for s in range(len(DNA[depth])-k+1):
                newPath = tuple([i for i in path] + [s])
                bestScore = exploreMotifs(DNA,k,newPath,bestScore)
            return bestScore

def BranchAndBoundMotifSearch(DNA, k):
    """ Finds a k-length motif within a list of DNA sequences"""
    global bestAlignment, prunedPaths
    bestAlignment = []
    prunedPaths = 0
    bestScore = 0
    bestScore = exploreMotifs(DNA,k,[],bestScore)
    print(f"{prunedPaths} paths were pruned")
    print("best motif start positions: ", bestAlignment, bestScore)
    for i in range(len(bestAlignment)):
        print(DNA[i][bestAlignment[i]:bestAlignment[i]+k])

%time BranchAndBoundMotifSearch(seqApprox[0:3], 10)
    

4781 paths were pruned
best motif start positions:  [17, 47, 18] 27
tagatctgaa
tggatccgaa
tagacccgaa
Wall time: 309 ms


In [15]:
seqs = [s.lower() for s in ["GAACTCATGGTG",
"AAAAGCACGGTC",
"TCAAAGCAAGGC",
"CCTAATCAGGGC",
"AAGTATGGACTC",
"ACTAAGCAGGGT",
"TCTCACGGCCCA",
"CCTCGTGGTGGG",
"TACCGTATGGTT",
"ACCACTCGTCGA"]]

print(seqs)
%time BranchAndBoundMotifSearch(seqs, 8)

['gaactcatggtg', 'aaaagcacggtc', 'tcaaagcaaggc', 'cctaatcagggc', 'aagtatggactc', 'actaagcagggt', 'tctcacggccca', 'cctcgtggtggg', 'taccgtatggtt', 'accactcgtcga']
208659 paths were pruned
best motif start positions:  [2, 2, 3, 3, 0, 3, 0, 0, 2, 3] 57
actcatgg
aagcacgg
aagcaagg
aatcaggg
aagtatgg
aagcaggg
tctcacgg
cctcgtgg
ccgtatgg
actcgtcg
Wall time: 4.51 s
